### general functions

In [ ]:
import numpy as np
np.set_printoptions(suppress=True, precision=2)

In [ ]:
def set_to_spec(lams):
    """
    switch to multiplicity notation and sort the eigenvalues
    """
    if isinstance(lams, list):
        lams = np.array(lams, dtype=float)
    lams_spec = np.unique(lams, return_counts=True)
    ind = np.argsort(lams_spec[0])
    lams_spec = (lams_spec[0][ind], lams_spec[1][ind])
    return lams_spec

def check_interlacing(lams, mus):
    """
    return lams and mus are interlacing or not
    """
    if isinstance(lams, list):
        lams = np.array(lams, dtype=float)
    if isinstance(mus, list):
        mus = np.array(mus, dtype=float)
    n = lams.shape[0]
    if mus.shape[0] != n-1:
        return False
    for i in range(n-1):
        if lams[i] > mus[i]:
            return False
        if mus[i] > lams[i+1]:
            return False
    return True

In [ ]:
set_to_spec([2,2,1,3])

In [ ]:
check_interlacing([1,3,5,7], [2,4,6])

### Path

In [ ]:
def lanczos(A, y):
    """
    Input: 
        - symmetrix matrix A
        - unit vector y
    
    Output:
        - Jacobi matrix J,
        - unitary matrix U such that A U = U J
    """
    n = A.shape[0]
    U = np.zeros((n,n), dtype=float)
    U[:,0] = y
    J = np.zeros((n,n), dtype=float)
    
    # i = 0
    Ay = A.dot(U[:,0])
    J[0,0] = U.T[0].dot(Ay)
    z = Ay - U.T[0] * J[0,0]
    J[1,0] = np.linalg.norm(z)
    J[0,1] = J[1,0]
    U[:,1] = z / J[1,0]
    
    # i = 1, ..., n-2
    for i in range(1,n-1):
        Ay = A.dot(U[:,i])
        J[i,i] = U.T[i].dot(Ay)
        z = Ay - U[:,i-1:i+1].dot(J[i-1:i+1,i])
        J[i+1,i] = np.linalg.norm(z)
        J[i,i+1] = J[i+1,i]
        U[:,i+1] = z / J[i+1,i]
    
    # i = n-1
    J[-1,-1] = A.dot(U[:,-1]).dot(U[:,-1])
        
    return J, U

def iep_path_y(lams, y):
    if isinstance(lams, list):
        lams = np.array(lams, dtype=float)
    if isinstance(y, list):
        y = np.array(y, dtype=float)
    D = np.diag(lams)
    return lanczos(D, y)

def iep_path(lams, mus):
    if isinstance(lams, list):
        lams = np.array(lams, dtype=float)
    if isinstance(mus, list):
        mus = np.array(mus, dtype=float)
    n = lams.shape[0]
    y = np.zeros((n,), dtype=float)
    for i in range(n):
        lam = lams[i]
        mus_prod = np.prod(mus - lam)
        lams_prod = np.prod(lams[np.arange(n) != i] - lam)
        y[i] = np.sqrt(mus_prod / lams_prod)
    return iep_path_y(lams, y)

In [ ]:
lams = np.array([1,2,3,4,5,6,7,8,9,10])
mus = lams[:-1] + 0.5
A = np.diag(lams)
y = np.array([1,1,1,1,1,1,1,1,1,1])
y = y / np.linalg.norm(y)

In [ ]:
J,U = lanczos(A, y)
print(J, U, sep="\n")
print(np.all(np.isclose(A.dot(U), U.dot(J))))
print(np.linalg.eigvalsh(J))

In [ ]:
J,U = iep_path_y(lams, y)
print(J, U, sep="\n")
print(np.all(np.isclose(A.dot(U), U.dot(J))))
print(np.linalg.eigvalsh(J))

In [ ]:
J,U = iep_path(lams, mus)
print(J, U, sep="\n")
print(np.all(np.isclose(A.dot(U), U.dot(J))))
print(np.linalg.eigvalsh(J))
print(np.linalg.eigvalsh(J[1:,1:]))

### Star

In [ ]:
def iep_star_strict(lams, mus):
    """
    Input:
        - lams: np.array of shape (n,) with distinct values
        - mus: np.array of shape(n-1,) with distinct values
    Output:
        - a bordered diagonal matrix A with spec(A) = lams, spec(A(1)) = mus
    """
    if isinstance(lams, list):
        lams = np.array(lams, dtype=float)
    if isinstance(mus, list):
        mus = np.array(mus, dtype=float)
    n = lams.shape[0]
    a1 = lams.sum() - mus.sum()
    ai = np.insert(mus, 0, a1)
    num = -np.prod(lams[:,np.newaxis] - mus, axis=0)
    gap_ind = np.array([
        [j for j in range(n-1) if j != i] for i in range(n-1)
    ])
    den = np.prod(mus[gap_ind].T - mus, axis = 0)
    bi = np.sqrt(num / den)
    A = np.zeros((n,n), dtype=float)
    A[np.arange(n), np.arange(n)] = ai
    A[0,1:] = bi
    A[1:,0] = bi
    return A

def iep_star(lams, mus):
    """
    Input:
        - lams: np.array of shape (n,)
        - mus: np.array of shape(n-1,), interlacing with lams
          if lams amd mus have any common value lam, 
          then mult_mus(lam) = mult_lams(lam) + 1.
    Output:
        - a bordered diagonal matrix A with spec(A) = lams, spec(A(1)) = mus
    Note:
        I am worried about numerical errors when determining common values.
    """
    if isinstance(lams, list):
        lams = np.array(lams, dtype=float)
    if isinstance(mus, list):
        mus = np.array(mus, dtype=float)
    if check_interlacing(lams, mus) == False:
        raise ValueError("Given lams and mus are not interlacing")
    n = lams.shape[0]
    lams_spec = set_to_spec(lams)
    mus_spec = set_to_spec(mus)
    common_align = ([], [])
    for i in range(lams_spec[0].shape[0]):
        for j in range(mus_spec[0].shape[0]):
            if np.isclose(lams_spec[0][i], mus_spec[0][j]):
                common_align[0].append(i)
                common_align[1].append(j)
    lams_p = np.delete(lams_spec[0], common_align[0])
    mus_p = mus_spec[0]
    A_p = iep_star_strict(lams_p, mus_p)
    ai = np.diag(A_p)
    bi = A_p[0, 1:]
    print("bi", bi)
    for k in range(len(common_align[0]) - 1, -1, -1):
        j = common_align[1][k] # off by one to indices of ai
        ai = np.insert(ai, j + 1, np.full(mus_spec[1][j] - 1, mus_spec[0][j]))
        old_b = bi[j]
        new_b = np.sqrt(old_b**2 / mus_spec[1][j])
        bi = np.insert(np.delete(bi, j), j, np.full(mus_spec[1][j], new_b))
    A = np.zeros((n,n), dtype=float)
    A[np.arange(n), np.arange(n)] = ai
    A[0,1:] = bi
    A[1:,0] = bi
    return A

In [ ]:
lams = [1,3,5,7,9]
mus = [2,4,6,8]
A = iep_star_strict(lams, mus)
print(A)
print(np.linalg.eigvalsh(A))
print(np.linalg.eigvalsh(A[1:,1:]))

In [ ]:
lams = [1,2,2,3,4,4,4,5]
mus = [2,2,2,4,4,4,4]
A = iep_star(lams, mus)
print(A)
print(np.linalg.eigvalsh(A))
print(np.linalg.eigvalsh(A[1:,1:]))

### Cycle

This section requires `iep_path_y` from the section of path.

In [ ]:
def cycle_ind(n, transpose=False):
    if transpose:
        return (list(range(1,n)) + [n-1], list(range(n-1)) + [0])
    else:
        return (list(range(n-1)) + [0], list(range(1,n)) + [n-1])

def floquet_multipliers(A):
    b1 = A[0,1]
    bn = A[0,-1]
    B = A[1:,1:]
    vals, vecs = np.linalg.eigh(B)
    vecs = vecs * np.sign(vecs[0])
    y,z = vecs[0], vecs[-1]
    rhos = - b1 / bn * y / z
    return rhos
    
def iep_cycle_arbm(a, rhos, b, mus):
    """
    Input:
        - a: value
        - rhos: np.array of shape(n-1,), entrywise sign (-1)**(n-j)
        - b: positive value
        - mus: np.array of shape(n-1,), distinct values
    Output:
        - a periodic Jacobi matrix A with 
            + tr(A) = a
            + Floquet multipliers = rhos (- b1 yj / bn zj)
            + b = b1 * ... * bn
            + mus = spec(A(1))
    """
    if isinstance(rhos, list):
        rhos = np.array(rhos, dtype=float)
    if isinstance(mus, list):
        mus = np.array(mus, dtype=float)
    n = rhos.shape[0] + 1
    
    # validate
    if not np.all(rhos * (-1)**np.arange(n-1, 0, -1) > 0):
        raise ValueError("rhos should be entrywise positive")
    if not b > 0:
        raise ValueError("b should be positive")
    if not np.unique(mus).shape == mus.shape:
        raise ValueError("mus contains distinct values")
        
    # construct matrix
    a1 = a - mus.sum()
    gap_ind = np.array([
        [j for j in range(n-1) if j != i] for i in range(n-1)
    ])
    Deltas = np.prod(mus[gap_ind].T - mus, axis = 0)
    b1 = np.sqrt((-1)**(n+1) * b * np.sum(rhos / Deltas))
    yjs = np.sqrt((-1)**(n+1) * b / b1**2 * rhos / Deltas)
    A = np.zeros((n,n), dtype=float)
    B,U = iep_path_y(mus, yjs)
    A[0,0] = a1
    A[[0,1],[1,0]] = b1
    A[1:,1:] = B
    bs = A[cycle_ind(n)]
    bn = b / np.prod(bs[:-1])
    A[[0,-1],[-1,0]] = bn
    return A

def iep_cycle_lbm(lams, b, mus):
    """
    Input:
        - lams: np.array of shape(n,)
        - b: positive value
        - mus: np.array of shape(n-1,)
        
    Note: 
        With phi_Lambda = prod_i (lam_i - x),
        we validate the data such that 
        (-1)**n phi_Lambda(mu_j) / b <= -4 if n - j is odd,
        (-1)**n phi_Lambda(mu_j) / b >= 0 if n - j is even.
        Necessarily, lams and mus are interlacing
        and mus is distinct.
        
    Output:
        - a periodic Jacobi matrix A with 
            + lams = spec(A)
            + b = b1 * ... * bn
            + mus = spec(A(1))
    """
    if isinstance(lams, list):
        lams = np.array(lams, dtype=float)
    if isinstance(mus, list):
        mus = np.array(mus, dtype=float)
    n = lams.shape[0]
    
    # validate
    phi_Lambda_mus = np.prod(lams[:,np.newaxis] - mus, axis=0)
    rprr = (-1)**n * phi_Lambda_mus / b + 2 # rho + 1/rho
    # fix numerical error around 2 and -2
    rprr[np.isclose(rprr, 2)] = 2
    rprr[np.isclose(rprr, -2)] = -2
    if not np.all(np.abs(rprr) >= 2):
        raise ValueError("absolute  value of rho + 1/rho should be >= 2")
    if not np.all(rprr * (-1)**np.arange(n-1, 0, -1) > 0):
        raise ValueError("sign of rho + 1/rho is wrong")
    if not b > 0:
        raise ValueError("b should be positive")
        
    # get rhos 
    # (we choose rhos in [-1,0) cup [1, inf), which is the larger solusion)
    rhos = (rprr + np.sqrt(rprr**2 - 4)) / 2
    a = lams.sum()
    return iep_cycle_arbm(a, rhos, b, mus)

def iep_cycle_l_auto(lams):
    """
    lams = (lam1, ..., lamn) satisfies
    ... < lam_{n-2} <= lam_{n-1} < lam_n
    Same as iep_cycle_lbm but choose b and mus automatically.
    """
    if isinstance(lams, list):
        lams = np.array(lams, dtype=float)
    n = lams.shape[0]

    mus = (lams[:-1] + lams[1:]) / 2
    phi_Lambda_mus = np.prod(lams[:,np.newaxis] - mus, axis=0)
    b = np.abs(phi_Lambda_mus[-1::-2]).min() / 4
    
    print(lams)
    print(mus)
    print(b)

    return iep_cycle_lbm(lams, b, mus)

In [ ]:
n = 5
A = np.zeros((n,n), dtype=float)
A[cycle_ind(n)] = 1
A[cycle_ind(n, transpose=True)] = 1

a = np.trace(A)
rhos = floquet_multipliers(A)
lams = np.linalg.eigvalsh(A)
b = np.prod(A[cycle_ind(n)])
mus = np.linalg.eigvalsh(A[1:,1:])

print("a", a)
print("rhos", rhos)
print("lams", lams)
print("b", b)
print("mus", mus)
print(A)

In [ ]:
A = iep_cycle_arbm(a, rhos, b, mus)

print("a", np.trace(A))
print("rhos", floquet_multipliers(A))
print("lams", np.linalg.eigvalsh(A))
print("b", np.prod(A[cycle_ind(n)]))
print("mus", np.linalg.eigvalsh(A[1:,1:]))
print(A)

In [ ]:
A = iep_cycle_lbm(lams, b, mus)

print("a", np.trace(A))
print("rhos", floquet_multipliers(A))
print("lams", np.linalg.eigvalsh(A))
print("b", np.prod(A[cycle_ind(n)]))
print("mus", np.linalg.eigvalsh(A[1:,1:]))
print(A)

In [ ]:
# lams = np.array([1,2,3,4,5])
lams = np.array([1,1,2,2,3])
A = iep_cycle_l_auto(lams)

print("a", np.trace(A))
print("rhos", floquet_multipliers(A))
print("lams", np.linalg.eigvalsh(A))
print("b", np.prod(A[cycle_ind(n)]))
print("mus", np.linalg.eigvalsh(A[1:,1:]))
print(A)